In [1]:
from src.model.multiple_output.convolution import Convolution1
from src.model.loss import pinball_loss
from src.trainers import compile_and_fit_with_pinball_loss
from src.loaders.window_generator import WindowGenerator
from src.loaders.data_loader import load_basic_preprocessed_train, load_submission_data
from src.make_submissions.load_test_features import make_submission_file

INPUT_STEPS = 48
OUTPUT_STEPS = 96
NUM_FEATURES = 11

train_df, valid_df, test_df = load_basic_preprocessed_train()

multi_window = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=INPUT_STEPS,
    label_width=OUTPUT_STEPS,
    shift=OUTPUT_STEPS,
)

conv = Convolution1(INPUT_STEPS, OUTPUT_STEPS, NUM_FEATURES)
submission_df = load_submission_data()
result_dict = {}


for i in range(1, 2):
    q = i/10
    loss_function = pinball_loss(q)
    
    compile_and_fit_with_pinball_loss(conv, multi_window, q)
    result_dict[q] = conv.evaluate(multi_window.test, verbose=0)
    make_submission_file(conv, submission_df, INPUT_STEPS, q)

ratio should be 3 length list, example: [0.6, 0.3, 0.1]
sum of ration should be 1, example: [0.7, 0.2, 0.1]
shape of train, valid, test: (36792, 11), (10511, 11), (5257, 11)
Epoch 1/1000
1146/1146 [==============================] - 8s 5ms/step - loss: 0.0713 - mean_absolute_error: 0.5363 - mean_squared_error: 0.7846 - _pinball_loss: 0.0713 - val_loss: 0.0579 - val_mean_absolute_error: 0.4737 - val_mean_squared_error: 0.7160 - val__pinball_loss: 0.0579
Epoch 2/1000
1146/1146 [==============================] - 5s 4ms/step - loss: 0.0507 - mean_absolute_error: 0.4325 - mean_squared_error: 0.6124 - _pinball_loss: 0.0507 - val_loss: 0.0560 - val_mean_absolute_error: 0.4660 - val_mean_squared_error: 0.7017 - val__pinball_loss: 0.0560
Epoch 3/1000
1146/1146 [==============================] - 5s 4ms/step - loss: 0.0493 - mean_absolute_error: 0.4208 - mean_squared_error: 0.6009 - _pinball_loss: 0.0493 - val_loss: 0.0548 - val_mean_absolute_error: 0.4514 - val_mean_squared_error: 0.6888 - val__p

In [3]:
sum([values[0] for values in result_dict.values()])
conv.evaluate(multi_window.test, verbose=0)[0]

0.04852638393640518

In [17]:
import src.model.multiple_output.convolution
import importlib
importlib.reload(src.model.multiple_output.convolution)

from src.model.multiple_output.convolution import Convolution2

one_week_window = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=48 * 7,
    label_width=48 * 2,
    shift=48 * 2,
)

conv1 = Convolution1(INPUT_STEPS, OUTPUT_STEPS, NUM_FEATURES)
compile_and_fit_with_pinball_loss(conv1, one_week_window, 0.1)
conv1.evaluate(one_week_window.test, verbose=0)[0]

Epoch 1/1000
1137/1137 [==============================] - 7s 6ms/step - loss: 0.0736 - mean_absolute_error: 0.5737 - mean_squared_error: 0.8122 - _pinball_loss: 0.0736 - val_loss: 0.0647 - val_mean_absolute_error: 0.5375 - val_mean_squared_error: 0.7727 - val__pinball_loss: 0.0647
Epoch 2/1000
1137/1137 [==============================] - 6s 5ms/step - loss: 0.0522 - mean_absolute_error: 0.4382 - mean_squared_error: 0.5806 - _pinball_loss: 0.0522 - val_loss: 0.0644 - val_mean_absolute_error: 0.5088 - val_mean_squared_error: 0.7529 - val__pinball_loss: 0.0644
Epoch 3/1000
1137/1137 [==============================] - 6s 5ms/step - loss: 0.0499 - mean_absolute_error: 0.4203 - mean_squared_error: 0.5606 - _pinball_loss: 0.0499 - val_loss: 0.0638 - val_mean_absolute_error: 0.5018 - val_mean_squared_error: 0.7387 - val__pinball_loss: 0.0638
Epoch 4/1000
1137/1137 [==============================] - 6s 5ms/step - loss: 0.0487 - mean_absolute_error: 0.4109 - mean_squared_error: 0.5511 - _pinball

InvalidArgumentError:  Input to reshape is a tensor with 16896 values, but the requested shape requires a multiple of 3696
	 [[node convolution2_1/sequential_4/lambda_4/Reshape (defined at C:\Users\bees1\PycharmProjects\dacon\235680\src\model\multiple_output\convolution.py:26) ]] [Op:__inference_test_function_336917]

Errors may have originated from an input operation.
Input Source operations connected to node convolution2_1/sequential_4/lambda_4/Reshape:
 IteratorGetNext (defined at <ipython-input-17-03c363424200>:18)

Function call stack:
test_function


In [18]:
conv2.evaluate(one_week_window.test, verbose=0)[0]

0.05603908747434616

In [4]:
import src.model.multiple_output.convolution
import importlib
importlib.reload(src.model.multiple_output.convolution)

from src.model.multiple_output.convolution import Convolution3

one_week_window = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=48 * 7,
    label_width=48 * 2,
    shift=48 * 2,
    label_columns=["TARGET"]
)

conv3 = Convolution3(INPUT_STEPS, OUTPUT_STEPS, NUM_FEATURES)
compile_and_fit_with_pinball_loss(conv3, one_week_window, 0.1)
conv3.evaluate(one_week_window.test, verbose=0)[0]

Epoch 1/1000
1137/1137 [==============================] - 14s 11ms/step - loss: 0.0762 - mean_absolute_error: 0.6100 - mean_squared_error: 0.9727 - _pinball_loss: 0.0762 - val_loss: 0.0762 - val_mean_absolute_error: 0.6295 - val_mean_squared_error: 1.0214 - val__pinball_loss: 0.0762
Epoch 2/1000
1137/1137 [==============================] - 10s 8ms/step - loss: 0.0565 - mean_absolute_error: 0.4737 - mean_squared_error: 0.6497 - _pinball_loss: 0.0565 - val_loss: 0.0735 - val_mean_absolute_error: 0.5812 - val_mean_squared_error: 0.9089 - val__pinball_loss: 0.0735
Epoch 3/1000
1137/1137 [==============================] - 9s 8ms/step - loss: 0.0529 - mean_absolute_error: 0.4418 - mean_squared_error: 0.5796 - _pinball_loss: 0.0529 - val_loss: 0.0796 - val_mean_absolute_error: 0.5627 - val_mean_squared_error: 0.8302 - val__pinball_loss: 0.0796
Epoch 4/1000
1137/1137 [==============================] - 8s 7ms/step - loss: 0.0501 - mean_absolute_error: 0.4163 - mean_squared_error: 0.5169 - _pinb

0.049442775547504425

In [2]:
import src.model.multiple_output.convolution
import importlib
importlib.reload(src.model.multiple_output.convolution)

from src.model.multiple_output.convolution import Convolution2

one_week_window = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=48 * 3,
    label_width=48 * 2,
    shift=48 * 2,
    label_columns=["TARGET"]
)

conv2 = Convolution2((3, 48), OUTPUT_STEPS, NUM_FEATURES)
compile_and_fit_with_pinball_loss(conv2, one_week_window, 0.1)
conv2.evaluate(one_week_window.test, verbose=0)[0]

Epoch 1/1000
1143/1143 [==============================] - 6s 5ms/step - loss: 0.0742 - mean_absolute_error: 0.5829 - mean_squared_error: 0.8323 - _pinball_loss: 0.0742 - val_loss: 0.0773 - val_mean_absolute_error: 0.6740 - val_mean_squared_error: 1.0914 - val__pinball_loss: 0.0773
Epoch 2/1000
1143/1143 [==============================] - 6s 5ms/step - loss: 0.0600 - mean_absolute_error: 0.4983 - mean_squared_error: 0.6755 - _pinball_loss: 0.0600 - val_loss: 0.0755 - val_mean_absolute_error: 0.6377 - val_mean_squared_error: 1.0115 - val__pinball_loss: 0.0755
Epoch 3/1000
1143/1143 [==============================] - 6s 5ms/step - loss: 0.0589 - mean_absolute_error: 0.4884 - mean_squared_error: 0.6630 - _pinball_loss: 0.0589 - val_loss: 0.0742 - val_mean_absolute_error: 0.6188 - val_mean_squared_error: 0.9786 - val__pinball_loss: 0.0742
Epoch 4/1000
1143/1143 [==============================] - 6s 5ms/step - loss: 0.0580 - mean_absolute_error: 0.4793 - mean_squared_error: 0.6513 - _pinball

0.043538033962249756

In [ ]:
import src.model.multiple_output.convolution
import importlib
importlib.reload(src.model.multiple_output.convolution)

from src.model.multiple_output.convolution import Convolution4

one_week_window = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=48 * 7,
    label_width=48 * 2,
    shift=48 * 2,
    label_columns=["TARGET"]
)

conv4 = Convolution4((3, 48), OUTPUT_STEPS, NUM_FEATURES)
compile_and_fit_with_pinball_loss(conv4, one_week_window, 0.1)
conv4.evaluate(one_week_window.test, verbose=0)[0]

Epoch 1/1000
1137/1137 [==============================] - 17s 14ms/step - loss: 0.0832 - mean_absolute_error: 0.6554 - mean_squared_error: 0.8883 - _pinball_loss: 0.0832 - val_loss: 0.0900 - val_mean_absolute_error: 0.6716 - val_mean_squared_error: 0.9053 - val__pinball_loss: 0.0900
Epoch 2/1000
1137/1137 [==============================] - 13s 11ms/step - loss: 0.0726 - mean_absolute_error: 0.5898 - mean_squared_error: 0.7381 - _pinball_loss: 0.0726 - val_loss: 0.0891 - val_mean_absolute_error: 0.7202 - val_mean_squared_error: 1.0367 - val__pinball_loss: 0.0891
Epoch 3/1000
1137/1137 [==============================] - 11s 9ms/step - loss: 0.0701 - mean_absolute_error: 0.5686 - mean_squared_error: 0.7059 - _pinball_loss: 0.0701 - val_loss: 0.0864 - val_mean_absolute_error: 0.6540 - val_mean_squared_error: 0.8960 - val__pinball_loss: 0.0864
Epoch 4/1000
1137/1137 [==============================] - 11s 10ms/step - loss: 0.0682 - mean_absolute_error: 0.5521 - mean_squared_error: 0.6811 - _

In [48]:
conv = Convolution4(INPUT_STEPS, OUTPUT_STEPS, NUM_FEATURES)
submission_df = load_submission_data()
result_dict = {}

one_week_window = WindowGenerator(
    train_df,
    valid_df,
    test_df,
    input_width=48 * 7,
    label_width=48 * 2,
    shift=48 * 2,
    label_columns=["TARGET"]
)


for i in range(1, 10):
    q = i/10
    print(f"q = {q}")
    conv = Convolution4(INPUT_STEPS, OUTPUT_STEPS, NUM_FEATURES)
    loss_function = pinball_loss(q)
    
    compile_and_fit_with_pinball_loss(conv, one_week_window, q)
    result_dict[q] = conv.evaluate(one_week_window.test, verbose=0)
    make_submission_file(conv, submission_df, 48 * 7, q)

q = 0.1
Epoch 1/1000
1137/1137 [==============================] - 12s 11ms/step - loss: 0.0843 - mean_absolute_error: 0.6627 - mean_squared_error: 0.9156 - _pinball_loss: 0.0843 - val_loss: 0.0995 - val_mean_absolute_error: 0.8767 - val_mean_squared_error: 1.3913 - val__pinball_loss: 0.0995
Epoch 2/1000
1137/1137 [==============================] - 12s 11ms/step - loss: 0.0727 - mean_absolute_error: 0.5909 - mean_squared_error: 0.7491 - _pinball_loss: 0.0727 - val_loss: 0.0994 - val_mean_absolute_error: 0.8103 - val_mean_squared_error: 1.2657 - val__pinball_loss: 0.0994
Epoch 3/1000
 438/1137 [==========>...................] - ETA: 6s - loss: 0.0711 - mean_absolute_error: 0.5754 - mean_squared_error: 0.7207 - _pinball_loss: 0.0711

KeyboardInterrupt: 

In [ ]:
submission_df.to_csv('conv4.csv')